In [ ]:
from OptiHPLCHandler import EmpowerHandler
from pprint import pprint
import sys

sys.path.insert(0, "../")

In [ ]:
# get api address from .env file
import os
from dotenv import load_dotenv

load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

Get instrument method

In [ ]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="XXXX", address=EMPOWER_API_ADDRESS)

handler.connection.default_get_timeout = 300
handler.connection.default_post_timeout = 300

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "@BSM_PDA_ELSDTemplate"
    full_method = handler.GetInstrumentMethod(method_name)

Original method name and gradient table

In [ ]:
print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

original_method_name = full_method.method_name

Generate ramp-up method

In [ ]:
from OptiHPLCHandler.applications.method_generators.ramp_method import generate_ramp_method
from OptiHPLCHandler.applications.revert_method import revert_method

generate_ramp_method(
    method=full_method,  # basis instrument method
    ramp_time=10,  # ramp time in minutes
    low_flow_rate=0.03,  # low flow rate in ml/min
    ramp_type="rampup",  # rampup or rampdown
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

Revert method to original method to generate other methods

In [ ]:
revert_method(method=full_method, original_method_name=original_method_name)
print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

In [ ]:
from OptiHPLCHandler.applications.method_generators.condense_gradient_table import (
    generate_condense_gradient_table,
)

generate_condense_gradient_table(
    method=full_method,  # basis instrument method
    new_method_time=10,  # time to condense gradient to
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

In [ ]:
from OptiHPLCHandler.applications.method_generators.alter_temperature import (
    generate_altered_temperature_method,
)

generate_altered_temperature_method(
    method=full_method,  # basis instrument method
    temperature_delta=5,  # adds temperature delta to current temperature, accepts negative values
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.column_temperature)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

In [ ]:
from OptiHPLCHandler.applications.method_generators.alter_strong_eluent_pct import (
    generate_altered_strong_eluent_method_pct,
)

generate_altered_strong_eluent_method_pct(
    method=full_method,  # basis instrument method
    strong_eluent_delta=1,  # adds strong eluent delta, accepts negative values
    maintain_wash_pct=True,  # false if the end of the gradient is the wash step
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

In [ ]:
from OptiHPLCHandler.applications.method_generators.add_isocratic_segment import (
    generate_add_isocratic_segment_to_method,
)

generate_add_isocratic_segment_to_method(
    method=full_method,  # basis instrument method
    isocratic_duration=10,  # duration of isocratic step
    index_of_isocratic_segment=0,  # where isocratic step is added, 0 for beginning, -1 for end
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

# revert
revert_method(method=full_method, original_method_name=original_method_name)